# Implementation of plain RNN

We loosly follow the imoplementation at https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/.

For our task we need to implement a network where it takes the current and previous inputs to predict the electrical output. If useful, we can include the previous electrical oputput as well.

For all the data we need to implement the difference, scale all and then train the model.

## Preliminaries

### Load libraries

In [1]:
#import own functions
import Global_Functions as gf
import Neuronal_Networks as nn

#import python standards
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import time
timestr = time.strftime("%Y-%m-%d_%H-%M/")

#import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
np.random.seed(123)

### Open Folder and Files

In [3]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [4]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [5]:
experiments = [ex_1, ex_4, ex_9, ex_20, ex_21, ex_22, ex_23, ex_24]
names = ['1', '4', '9', '20', '21','22', '23', '24']

In [6]:
OPEN_SYNTHETIC = 'C:/Users/FlorianLeiser/Documents/Masterarbeit/Data/Synthetic_Data/Without_Noise/Sample_Ratio_1/quadratic_40_0_linear_1'

In [7]:
experiments = gf.load_synthetic(OPEN_SYNTHETIC, length = 200)
names = [str(x+1) for x in range(len(experiments)-1)]
names.append("hand")

### Specify parameters

In [8]:
NEUR = 512
EPOCH = 200
LAG = 60
DIFFERENCE = None
BATCH_SIZE = 512

In [9]:
train = "150"
val = "30"

train_exs = experiments[:150]
ex_train = gf.use_multiple_experiments(train_exs)
ex_val = gf.use_multiple_experiments(experiments[150:180])
ex_test = gf.use_multiple_experiments(experiments[180:200])

In [10]:
image_path = "../Images/Validation/Synthetic/RNN/" + timestr
image_folder = image_path
gf.check_folder(image_folder)

Creation of directory ../Images/Validation/Synthetic/RNN/2021-05-28_11-31/ successful.


In [11]:
model_path = "../Models/Validation/Synthetic/RNN/" + timestr
model_folder = model_path
gf.check_folder(model_folder)

Creation of directory ../Models/Validation/Synthetic/RNN/2021-05-28_11-31/ successful.


In [12]:
specs = {
    'neurons': NEUR,
    'epochs': EPOCH,
    'lag': LAG,
    'difference': DIFFERENCE,
    'batch_size': BATCH_SIZE,
    'model': "LSTM(Neur), Dense(Neur/2), Dense(Out)",
    'data': 'Synthetic - (40, 0, 1)',
    'data_points': '150 for training - 30 for validation',
    'sampling_rate': 1,
    'loss_function': 'Adagrad'
}

specs_str = ""
for cat, descr in specs.items():
    specs_str += f'{cat}: {descr} \n'

with open(image_folder + "00_specs.txt", "w") as text_file:
    text_file.write(specs_str)

## Train model

We use scaling and the fitting method which both are in the Neural Network file in the utilities folder.

In [13]:
model, history, scaler, X_train, y_train, _, X_val, y_val = nn.train_model(ex_train, ex_val,
                                                                           difference_chosen = DIFFERENCE,
                                                                           save_folder = model_folder,
                                                                           lag_chosen=LAG, batch_size = BATCH_SIZE,
                                                                           neurons_chosen=NEUR, nmb_epochs=EPOCH)

Folder already exists.
Epoch 10 of 200 is done.
Epoch 20 of 200 is done.
Epoch 30 of 200 is done.
Epoch 40 of 200 is done.
Epoch 50 of 200 is done.
Epoch 60 of 200 is done.
Epoch 70 of 200 is done.
Epoch 80 of 200 is done.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

In [ ]:
all_scaler = list()
all_X_values = list()
all_y_values = list()
all_predictions = list()
all_scaled_predictions = list()
results = pd.DataFrame(index = names, columns = ['rmse', 'r2', 'mae', 'maxae', 'rmse_scaled', 'r2_scaled', 'mae_scaled', 'maxae_scaled'])

for i in range(len(experiments)):
    rms, r2, mae, maxae = [], [], [], []
    scaler, X, y, preds_scaled, preds = nn.predictions(experiments[i], model,
                                                       difference_chosen = DIFFERENCE,
                                                      lag_chosen = LAG,
                                                      batch_size = BATCH_SIZE)
    all_scaler.append(scaler)
    all_X_values.append(X)
    all_y_values.append(y)
    all_predictions.append(preds)
    all_scaled_predictions.append(preds_scaled)
    
    gf.create_prediction_plot(experiments[i]['el_power'], preds, image_folder,
                              title = 'Predictions using plain RNN ',
                             specs = 'on ex_{0} with model trained on {1}'.format(names[i], train))
    
    results_ex = gf.measure_difference(experiments[i]['el_power'], preds, should_print = False)
    rms.append(results_ex['RMSE'][0])
    r2.append(results_ex['R2'][0])
    mae.append(results_ex['MAE'][0])
    maxae.append(results_ex['MaxAE'][0])
                   
    results.loc[names[i], 'rmse'] = rms
    results.loc[names[i], 'r2'] = r2
    results.loc[names[i], 'mae'] = mae
    results.loc[names[i], 'maxae'] = maxae
    results.to_csv(image_folder + "99_results.csv", sep = "|", encoding = 'utf-8')

In [ ]:
gf.save_losses(history, image_folder)